In [1]:
import numpy as np
np.object = object
np.int = int
np.float = float
np.bool = bool

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection
import re
     

In [3]:
data_df = pd.read_csv('D:\dialogs_expanded.csv\dialogs_expanded.csv', index_col=False)
data_df

,Unnamed: 0,question,answer,question_as_int,answer_as_int,question_len,answer_len
0,1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....,"[54, 67, 74, 74, 12, 1, 40, 1, 82, 70, 77, 83,...","[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...",71,55
1,2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...,"[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...","[46, 73, 63, 87, 14, 14, 14, 1, 82, 70, 67, 76...",55,73
2,3,You're asking me out. That's so cute. What's ...,Forget it.,"[56, 77, 83, 8, 80, 67, 1, 63, 81, 73, 71, 76,...","[37, 77, 80, 69, 67, 82, 1, 71, 82, 14]",62,10
3,4,"No, no, it's my fault -- we didn't have a prop...",Cameron.,"[45, 77, 12, 1, 76, 77, 12, 1, 71, 82, 8, 81, ...","[34, 63, 75, 67, 80, 77, 76, 14]",65,8
4,9,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.,"[38, 77, 81, 70, 12, 1, 71, 68, 1, 77, 76, 74,...","[43, 67, 82, 1, 75, 67, 1, 81, 67, 67, 1, 85, ...",46,25
...,...,...,...,...,...,...,...
139404,221608,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...,"[54, 67, 74, 74, 1, 82, 70, 63, 82, 1, 77, 76,...","[68, 82, 1, 65, 77, 83, 74, 66, 1, 64, 67, 1, ...",47,59
139405,221609,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...,"[34, 70, 77, 77, 81, 67, 1, 87, 77, 83, 80, 1,...","[42, 67, 67, 78, 1, 81, 82, 67, 63, 66, 87, 14...",61,85
139406,221610,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ...","[34, 77, 74, 77, 76, 67, 74, 1, 35, 83, 80, 76...","[38, 77, 77, 66, 1, 77, 76, 67, 81, 12, 1, 87,...",74,60
139407,221611,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...,"[56, 77, 83, 80, 1, 77, 80, 66, 67, 80, 81, 12...","[40, 8, 75, 1, 82, 77, 1, 82, 63, 73, 67, 1, 8...",24,56


In [4]:
data_df.drop(['Unnamed: 0','question_as_int','answer_as_int','question_len','answer_len'],axis=1,inplace=True)

In [5]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139409 entries, 0 to 139408
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  139409 non-null  object
 1   answer    139409 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [6]:
data_df

,question,answer
0,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
1,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
2,You're asking me out. That's so cute. What's ...,Forget it.
3,"No, no, it's my fault -- we didn't have a prop...",Cameron.
4,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.
...,...,...
139404,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...
139405,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...
139406,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ..."
139407,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...


In [7]:
def clean_text(text): #---------- defult cleaning function to use any time ...
    text = text.lower()
    text = re.sub('\[.*?\]',' ',text)
    text = re.sub('https?://\S+|www\.\.S+',' ',text)
    text = re.sub('<.*?>+',' ',text)
    text = re.sub('\n',' ',text)
    text = re.sub('[^\w]',' ',text)
    text = re.sub('\w*\d\w*',' ',text)
    return text

data_df.question = data_df.question.map(clean_text)
data_df.answer = data_df.answer.map(clean_text)

In [8]:
def add_start_end(text):
    text = f'<start> {text} <end>'
    return text

data_df.question = data_df.question.map(add_start_end)
data_df.answer = data_df.answer.map(add_start_end)

In [9]:
data_df

,question,answer
0,<start> well i thought we d start with pronun...,<start> not the hacking and gagging and spitti...
1,<start> not the hacking and gagging and spitti...,<start> okay then how bout we try out some...
2,<start> you re asking me out that s so cute ...,<start> forget it <end>
3,<start> no no it s my fault we didn t hav...,<start> cameron <end>
4,<start> gosh if only we could find kat a boyf...,<start> let me see what i can do <end>
...,...,...
139404,<start> well that one the one who keeps looki...,<start> ft could be you flatter yourself coghi...
139405,<start> choose your targets men that s right ...,<start> keep steady you re the best shots of ...
139406,<start> colonel durnford william vereker i...,<start> good ones yes mr vereker gentlemen ...
139407,<start> your orders mr vereker <end>,<start> i m to take the sikali with the main c...


In [10]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;=?@[\]^_`{|}~\t\n', oov_token='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [11]:
question_sequence, question_tokenizer = tokenize(data_df.question)
answer_sequence, answer_tokenizer = tokenize(data_df.answer)

In [12]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(question_sequence, 
                answer_sequence, test_size = 0.1, random_state=42) 

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((125468, 29), (13941, 29), (125468, 32), (13941, 32))

In [13]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print('%d---> %s' % (t, lang.index_word[t]))

print('Question')
convert(question_tokenizer, x_train[0])
print()
print('Answer')
convert(answer_tokenizer, y_train[0])

Question
2---> <start>
80---> yeah
13---> that
8---> s
11004---> blush
36---> on
32---> my
301---> wife
3384---> uses
9---> it
3---> <end>

Answer
2---> <start>
204---> ask
5535---> travis
22---> he
7---> s
6---> the
1765---> ladies
104---> man
3---> <end>


In [14]:
vocab_inp_size = len(question_tokenizer.word_index)+1
vocab_tar_size =  len(answer_tokenizer.word_index)+1
embedding_dim = 256
units = 1024
batch_size=32

In [15]:
def create_dataset(x, y, batch_size=32):
    
    data = tf.data.Dataset.from_tensor_slices((x, y))

    data = data.shuffle(1028)
    data = data.batch(batch_size, drop_remainder=True)

    data = data.prefetch(tf.data.experimental.AUTOTUNE)

    return data

train_dataset = create_dataset(x_train, y_train)
test_dataset = create_dataset(x_test, y_test)

In [16]:
for q, a in train_dataset.take(1):
    print(f'Question:{q.shape}\n{q}')
  
    print(f'Answer:{a.shape}\n{a}')

Question:(32, 29)
[[    2    12    29     4   111    44    19     6  3578  1043  3003     3
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    17    28    10   203 20265     5    23   415   131     3     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    39    86  7134     3     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2     5   308     9     9     8    39    31    47     3     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    29    43   146    53    47     3     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    2    67   145     9    35   150   131   143   106   148   317    22
   

In [17]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
        super(Encoder, self).__init__()

        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.gru = tf.keras.layers.GRU(self.encoder_units, 
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer = 'glorot_uniform')

    def call(self, x, hidden):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoder_units))

In [18]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size):
        super(Decoder, self).__init__()

        self.batch_size = batch_size
        self.decoder_units = decoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.gru = tf.keras.layers.GRU(self.decoder_units, 
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer = 'glorot_uniform')
      
        self.fc = tf.keras.layers.Dense(vocab_size)


    def call(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.gru(x, initial_state = hidden)
        output = tf.reshape(output, (-1, output.shape[2]))
        x =  tf.nn.softmax(self.fc(output))
        return x, hidden

In [19]:
# vocab_inp_size = len(eng_tokenizer.word_index)+1
# vocab_tar_size =  len(spn_tokenizer.word_index)+1
# embedding_dim = 256
# units = 1024
# batch_size=32

encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(q, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (32, 29, 1024)
Encoder Hidden state shape: (batch size, units) (32, 1024)


In [20]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, batch_size)

sample_decoder_output, _ = decoder(tf.random.uniform((batch_size, 1)), sample_hidden)

print ('Decoder output shape: (batch size, vocab_size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch size, vocab_size) (32, 27850)


In [21]:
# create the optimizer using the Adam optimizer
optimizer = tf.keras.optimizers.Adam()
# create the loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction='none')

# define the loss function for the training
def loss_function(real, pred):
    # create the mask to ignore the padding tokens
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # mask shape == (batch_size, sequence_length)
    # calculate the loss
    loss_ = loss_object(real, pred)
    # mask the loss
    # how the mask works:
    # if the value is 1, the loss is calculated
    # if the value is 0, the loss is ignored
      #[1,1,1,1,1,1,0,0,0,0,0] mask
      # *
      #[2,6,2,1,6,3,2,1,5,7,9] input
      # =
      #[2,6,2,1,6,3,0,0,0,0,0] output
    mask = tf.cast(mask, dtype=loss_.dtype)
    # mask shape == (batch_size, sequence_length)

    loss_ *= mask
    # calculate the average loss per batch 
    return tf.reduce_mean(loss_)

In [22]:
# create the training metric 
train_loss = tf.metrics.Mean(name='train loss')
# create the testing metric 
test_loss =tf.metrics.Mean(name='test loss')

In [23]:
@tf.function
def train_step(inputs, target, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inputs, enc_hidden)
        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([answer_tokenizer.word_index['']] * inputs.shape[0], 1)

        for t in range(1, target.shape[1]):
            predictions, dec_hidden = decoder(dec_input, dec_hidden)
            loss += loss_function(target[:, t], predictions)

            dec_input = tf.expand_dims(target[:, t], 1)

        batch_loss = (loss / int(target.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(batch_loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    train_loss(batch_loss)
    return batch_loss

In [24]:
# create the training step
# using the tf.function decorator to speed up the training process by converting the training function to a TensorFlow graph
@tf.function 
def test_step(inputs, target, enc_hidden):
    # the encoder_hidden is the initial hidden state of the encoder
    # enc_hidden shape == (batch_size, hidden_size)
    # inilaize the loss to zero
    loss = 0
    # pass the input to the encoder 
    # enc_output shape == (batch_size, 49, hidden_size) 
    # enc_hidden shape == (batch_size, hidden_size)
    # using the encoder to get the encoder_output and the encoder_hidden
    enc_output, enc_hidden = encoder(inputs, enc_hidden)
    # set the initial decoder hidden state to the encoder hidden state
    dec_hidden = enc_hidden
    # create the start token
    # start_token shape == (batch_size, 1)
    # repeat the start token for the batch size times
    dec_input = tf.expand_dims([answer_tokenizer.word_index['']] * inputs.shape[0], 1)
    for t in range(1, target.shape[1]):
        # passing enc_output to the decoder with dec_hidden as the initial hidden state
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        # calculate the loss for the current time step using the loss function 
        loss += loss_function(target[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(target[:, t], 1)
    # calculate the loss for the current batch
    batch_loss = (loss / int(target.shape[1]))
    # add the batch loss to the test loss metric
    test_loss(batch_loss)

In [25]:

# set the epochs to 10
EPOCHS = 10
# set the old test loss to high number 

old_test_loss=1000000
# create the training loop
for epoch in range(EPOCHS):
    # reset the training loss metric
    train_loss.reset_states()
    # reset the testing loss metric
    test_loss.reset_states()

    # initalize the hidden state of the encoder to zeros 
    enc_hidden = encoder.initialize_hidden_state()
    # create the training progress bar set the total number of batches to the length of the training dataset and the batch size to the test size
    steps_per_epoch = answer_sequence.shape[0]//batch_size #=> 4356 batch in the dataset 
    bar = tf.keras.utils.Progbar(target=steps_per_epoch)
    
    count=0
    # iterate over the training dataset 
    for (batch, (inputs, target)) in enumerate(train_dataset):
        # update the progress bar
        count += 1
        # run the training step
        batch_loss = train_step(inputs, target, enc_hidden)
        bar.update(count)  # manually update the progress bar
                                                  
    
         
    
    # iterate over the testing dataset    
    for (batch, (inputs, target)) in enumerate(test_dataset):
        count += 1
        # run the testing step
        batch_loss = test_step(inputs, target, enc_hidden)
        bar.update(count)
    # save the best performance model on the test dataset 
    
    if old_test_loss> test_loss.result():
        # set the old test loss to the test loss 
        old_test_loss= test_loss.result()
        encoder.save(filepath='/content/models/encoder')
        decoder.save(filepath='/content/models/decoder')
        print('Model is saved')
    # print the training and testing loss
    print('#' * 50)
    print(f'Epoch #{epoch + 1}')
    print(f'Training Loss {train_loss.result()}')
    print(f'Testing Loss {test_loss.result()}')
    print('#' * 50)
  

4355/4356 [============================>.] - ETA: 0s

INFO:tensorflow:Assets written to: /content/models/encoder\assets


INFO:tensorflow:Assets written to: /content/models/encoder\assets


INFO:tensorflow:Assets written to: /content/models/decoder\assets


INFO:tensorflow:Assets written to: /content/models/decoder\assets


Model is saved
##################################################
Epoch #1
Training Loss 1.3988901376724243
Testing Loss 1.2896350622177124
##################################################
 209/4356 [>.............................] - ETA: 18:21

KeyboardInterrupt: 

In [26]:
# create the chatbot function
# the chatbot function takes in the question as input and answers the input sentence 
def chatbot(sentence):
  
  # clean the input question sentence 
  sentence = clean_text(sentence)
  # add the start token to the sentence
  sentence =add_start_end(sentence)
  # tokenize the sentence
  inputs = question_tokenizer.texts_to_sequences([sentence])
  # pad the sentence
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                         maxlen=29,
                                                         padding='post')
  
  # initalize the hidden state of the encoder to zeros
  hidden = [tf.zeros((1, units))]
  # pass the sentence to the encoder with the hidden state as the initial hidden state
  enc_out, enc_hidden = encoder(inputs, hidden)
  # set the initial decoder hidden state to the encoder hidden state
  dec_hidden = enc_hidden
  # create the start token
  # start_token shape == (batch_size, 1)
  # repeat the start token for the batch size times
  dec_input = tf.expand_dims([answer_tokenizer.word_index['']], 0)
  # create the result string
  result = ''
  # loop over the length of the sentence (32)

  for t in range(32):
    # passing the encoder output and the decoder hidden state to the decoder make sure the decoder input is the previous predicted word
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # getting the predicted word index
    predicted_id = tf.argmax(predictions[0]).numpy()
    # getting the predicted word using the predicted index
    # add the predicted word to the result string 
    result += answer_tokenizer.index_word[predicted_id] + ' '
    # if the predicted word is the  token then stop the loop
    if answer_tokenizer.index_word[predicted_id] == '':
      # remove the  and  tokens from the result string
      result = result.replace(' ', '')
      result = result.replace('  ','')
      # remove the  and  tokens from the sentence string
      sentence = sentence.replace(' ', '')
      sentence = sentence.replace(' ', '')
      return  sentence, result

    # using the predicted word as the next decoder input
    dec_input = tf.expand_dims([predicted_id], 0)
  # remove the  and  tokens from the result string
  result = result.replace(' ', '')
  result = result.replace('','')
  # remove the  and  tokens from the sentence string
  sentence = sentence.replace(' ', '')
  sentence = sentence.replace('', '')
  

  
 
  
  # return the result string and the original sentence
  return sentence, result

In [32]:
chatbot("can you run ?")

('<start>canyourun<end>',
 'imnotsureimnotsure<end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end>')

In [33]:
chatbot('what is the weather outside')

('<start>whatistheweatheroutside<end>',
 'imnotsure<end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end><end>')